# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [76]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [78]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wellawaya,6.7318,81.1018,25.37,65,100,1.05,LK,1724789307
1,1,awjilah,29.1081,21.2869,33.47,25,73,6.01,LY,1724789308
2,2,wailua homesteads,22.0669,-159.3780,27.44,72,0,6.69,US,1724789310
3,3,waitangi,-43.9535,-176.5597,12.12,98,100,3.58,NZ,1724789311
4,4,khvoynaya,58.9000,34.5333,14.65,72,100,1.96,RU,1724789312


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [80]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    size='Humidity', color='Humidity', cmap='viridis', 
    alpha=0.6, hover_cols=['City', 'Humidity'], geo=True, 
    tiles='CartoLight', frame_width=800, frame_height=600)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [81]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 27) & 
                                 (city_data_df['Max Temp'] > 21) & 
                                 (city_data_df['Wind Speed'] < 4.5) & 
                                 (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df.head()

C:\Users\tasee\AppData\Local\Temp\ipykernel_13040\3792964106.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,kolonia,50.9333,6.9500,22.20,65,0,2.06,DE,1724789337
53,53,colonia,50.9333,6.9500,22.20,65,0,2.06,DE,1724789370
89,89,toliara,-23.3500,43.6667,22.14,64,0,1.42,MG,1724789413
186,186,dubovka,49.0583,44.8290,21.14,47,0,4.36,RU,1724789522
189,189,tafi viejo,-26.7320,-65.2592,21.67,24,0,2.26,AR,1724789526


### Step 3: Create a new DataFrame called `hotel_df`.

In [82]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,kolonia,DE,50.9333,6.9500,65,
53,colonia,DE,50.9333,6.9500,65,
89,toliara,MG,-23.3500,43.6667,64,
186,dubovka,RU,49.0583,44.8290,47,
189,tafi viejo,AR,-26.7320,-65.2592,24,
219,san sebastian de la gomera,ES,28.0916,-17.1133,78,
284,santa cruz,BO,-17.8000,-63.1667,25,
324,al qusayr,SY,34.5091,36.5798,68,
371,old town,CZ,50.0870,14.4202,63,
407,adler,RU,43.4300,39.9200,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [87]:
# Set parameters to search for a hotel
radius = 10000 #metres
params = {
    "categories": "accommodation.hotel", 
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")
tracker = 0
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude= row["Lng"]

    # print('lat', latitude)
    # print('lon', longitude)
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    # print(params)
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)


    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kolonia - nearest hotel: Wasserturm Hotel Cologne
colonia - nearest hotel: Wasserturm Hotel Cologne
toliara - nearest hotel: Ambary
dubovka - nearest hotel: Алиса
tafi viejo - nearest hotel: Casa Calchaqui
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
santa cruz - nearest hotel: Aviador Hotel Boutique
al qusayr - nearest hotel: No hotel found
old town - nearest hotel: Terasa U Prince
adler - nearest hotel: Shine
argo - nearest hotel: Apollon Hotel
nchelenge - nearest hotel: No hotel found
almazan - nearest hotel: Hotel Antonio
pacific grove - nearest hotel: Pacific Grove Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
24,kolonia,DE,50.9333,6.9500,65,Wasserturm Hotel Cologne
53,colonia,DE,50.9333,6.9500,65,Wasserturm Hotel Cologne
89,toliara,MG,-23.3500,43.6667,64,Ambary
186,dubovka,RU,49.0583,44.8290,47,Алиса
189,tafi viejo,AR,-26.7320,-65.2592,24,Casa Calchaqui
219,san sebastian de la gomera,ES,28.0916,-17.1133,78,Hotel Villa Gomera
284,santa cruz,BO,-17.8000,-63.1667,25,Aviador Hotel Boutique
324,al qusayr,SY,34.5091,36.5798,68,No hotel found
371,old town,CZ,50.0870,14.4202,63,Terasa U Prince
407,adler,RU,43.4300,39.9200,83,Shine


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [88]:
%%capture --no-display

# Configure the map plot

hotel_df['category'] = hotel_df['City']  # or any other column you want to use for categories

# Plot the map with points colored by the 'category' column
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',          # Longitude and Latitude columns
    geo=True,               # Enables geographic plotting
    tiles='OSM',            # Sets the background map to OpenStreetMap
    c='category',           # Color points by the category column
    size=25,                # Size of the points on the map
    hover_cols=['Hotel Name', 'City', 'Country'],  # Include additional information in hover tooltips
).opts(width=800, height=600)  # Adjust the size of the plot

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (category,Hotel Name,City,Country)